In [1]:
import plotly.graph_objects as go
import pandas as pd
import math
large_presentation = dict(layout=go.Layout(title_font=dict(family="presentation", size=24)))

In [2]:
# $SPY Dataframe 2005-04-18 to 2020-04-16
df = pd.read_csv("./CSV/SPYYahoo.csv")
df.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2005-04-18,114.120003,114.959999,113.959999,114.500000,84.596092,100035200
1,2005-04-19,115.099998,115.529999,114.830002,115.410004,85.268448,64930100
2,2005-04-20,115.379997,115.629997,113.550003,113.800003,84.078949,107735900


In [3]:
df.set_index(pd.to_datetime(df['Date']), inplace = True)
df.drop(columns = ["Date"], inplace = True)

In [6]:
def algo(ticker_symbol, moving_A, moving_B, initial_shares):
    
    # Initial Inputs
    value = []
    shares = initial_shares
    cash = 0
    
    # Moving Averages of Closing Price
    A = ticker_symbol.Close.rolling(moving_A).mean()
    B = ticker_symbol.Close.rolling(moving_B).mean()
    
    # Adding NaN values to the list based on moving_A
    [value.append(math.nan) for i in range(moving_A)]
        
    # Moving Average Algorithm
    for i, price in enumerate(ticker_symbol.Close[moving_A:]):
        if (A[(moving_A - 1) + i] < B[(moving_B - 1) + i]) and (shares > 0):
            # Sell Shares
            cash = cash + (shares * price)
            shares = 0
            value.append(cash)

        elif (A[(moving_A - 1) + i] > B[(moving_B - 1) + i]) and (cash > price):
            # Buy Shares
            shares = shares + (cash//price)
            cash = cash - (price * (cash//price))
            value.append(cash + (shares * price))
            
        else:
            value.append(cash + (shares * price))
        
    # Saving to new DataFrame
    df["steady_state"] = df["Close"] * initial_shares
    df["value"] = value
    df["p_l_steady_state"] = df["steady_state"].diff()
    df["p_l_trading"] = df["value"].diff()
    df['p_l_diff'] = df["p_l_steady_state"] - df["p_l_trading"]
    
    # Display Plots
    
    # Value Trend
    fig = go.Figure(data = [
    go.Scatter(x = df.index, y = df['steady_state'], mode = 'lines', name = f"{initial_shares} Shares"),
    go.Scatter(x = df.index, y = df['value'], mode = 'lines', name = f"{moving_A}:{moving_B} Trading")
    ])

    fig.update_layout(
        template = large_presentation
    )

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    
    fig.show()
    
    # Profit/Loss Graph
    plot = go.Figure(data = [
    go.Scatter(x = df.index, y = df['p_l_steady_state'], mode = 'lines', name = "Steady State Profit/Loss"),
    go.Scatter(x = df.index, y = df['p_l_trading'], mode = 'lines', name = "Trading Profit/Loss", opacity = 0.55),
    go.Scatter(x = df.index, y = df['p_l_diff'], mode = 'lines', name = "Difference in Profit/Loss", opacity = 0.4)
    ])

    plot.update_layout(
        title = 'Profit/Loss',
        template = large_presentation
    )

    plot.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    
    plot.show()
    return 

In [7]:
algo(df, 200, 50, 1000)